In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.80 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.78 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  35.70 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  44.51 MB/s


In [2]:
import pandas as pd
import folium

In [3]:
# The code was removed by Watson Studio for sharing.

First a GET request for FourSquare API is constructed. Here, we find coffee near Helsinki center, in 1000 m radius. In general it's better to let requests to handle adding parameters to url than manually construct it.

In [4]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Helsinki",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

Parsing response to JSON. We get to following keys in our dictionary:

In [5]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedBounds', 'suggestedFilters', 'totalResults', 'headerFullLocation', 'headerLocationGranularity', 'groups', 'query', 'geocode', 'headerLocation'])

Most of the items on dictionary are giving some general information like the name of the city, it's geo code and so on. `groups` contains the actual search result.

In [6]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Helsinki', 'Helsinki')

In [7]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 60.17817637760799, 'lng': 24.952162607605953},
  'sw': {'lat': 60.16028899950671, 'lng': 24.921636447256862}},
 93)

In [8]:
d["geocode"]

{'cc': 'FI',
 'center': {'lat': 60.16952, 'lng': 24.93545},
 'displayString': 'Helsinki, Finland',
 'geometry': {'bounds': {'ne': {'lat': 60.29788198177167,
    'lng': 25.2415192349548},
   'sw': {'lat': 60.12648197436881, 'lng': 24.8240229894074}}},
 'longId': '72057594038586161',
 'slug': 'helsinki-finland',
 'what': '',
 'where': 'helsinki'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is `data.json()["response"]["groups"][0]["items"]`.

Let's print couple of first items to get again better idea of the structure of the data:

In [9]:
d["groups"][0].keys()

dict_keys(['name', 'type', 'items'])

In [10]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [11]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-5-4ddd2d44b0fba481fc927360-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'primary': True,
    'shortName': 'Bakery'}],
  'id': '4ddd2d44b0fba481fc927360',
  'location': {'address': 'Yrjönkatu 25',
   'cc': 'FI',
   'city': 'Helsinki',
   'country': 'Suomi',
   'formattedAddress': ['Yrjönkatu 25', '00100 Helsinki', 'Suomi'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 60.16789868904846,
     'lng': 24.938189914522386}],
   'lat': 60.16789868904846,
   'lng': 24.938189914522386,
   'postalCode': '00100',
   'state': 'Uusimaa'},
  'name': 'Patisserie Teemu & Markus',
  'photos': {'count': 0, 'groups': []}}}

In [12]:
items[1]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-5-50f688d5e4b023d2f274b506-1',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'primary': True,
    'shortName': 'Coffee Shop'}],
  'id': '50f688d5e4b023d2f274b506',
  'location': {'address': 'Fredrikinkatu 59',
   'cc': 'FI',
   'city': 'Helsinki',
   'country': 'Suomi',
   'crossStreet': 'Kansakoulukatu',
   'formattedAddress': ['Fredrikinkatu 59 (Kansakoulukatu)',
    '00100 Helsinki',
    'Suomi'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 60.167580051384675,
     'lng': 24.932525558737044}],
   'lat': 60.167580051384675,
   'lng': 24.932525558737044,
   'postalCode': '00100',
   'state': 'Uusimaa'},
  'name': 'Kaffecentralen',
 

Based on this, a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [13]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 49 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,4ddd2d44b0fba481fc927360,Patisserie Teemu & Markus,Bakery,Yrjönkatu 25,00100,60.167899,24.938190
1,50f688d5e4b023d2f274b506,Kaffecentralen,Coffee Shop,Fredrikinkatu 59,00100,60.167580,24.932526
2,5aec747112f0a9002c9b92ab,La Torrefazione,Café,Mannerheimintie 22,00100,60.170721,24.936158
3,4b4cb879f964a520c0bb26e3,The Ounce,Tea Room,Fredrikinkatu 55,00100,60.167182,24.932993
4,4af1c9e2f964a52031e321e3,La Torrefazione,Coffee Shop,Aleksanterinkatu 50,00100,60.168877,24.943845


Some density based estimator is giving a good tip where to start a new coffee business. There's a `HeatMap` plugin ready in Folium, let's use that, and visualize all the existing Cafes to same map:

In [14]:
helsinki_center = d["geocode"]["center"]
helsinki_center

{'lat': 60.16952, 'lng': 24.93545}

In [15]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_helsinki = folium.Map(location=[helsinki_center["lat"], helsinki_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_helsinki)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_helsinki.add_child(plugins.HeatMap(hm_data))

map_helsinki

## Results

Based on these results, one possibly good location for new Cafe would be in crossroad of Aleksanterinkatu and Mikonkatu:

In [16]:
lat = 60.168749
lng = 24.945747
map_helsinki = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_helsinki)
map_helsinki